In [ ]:
import cv2
import mediapipe as mp
import math
import os
import json
from google.colab import drive

# 1. Connexion au Google Drive
drive.mount('/content/drive')

# 2. Chemin vers votre dossier sur le Drive
# Si votre dossier est à la racine du Drive, le chemin est celui-ci :
ROOT_DRIVE = '/content/drive/MyDrive/coach_dataset'

# --- UTILS MATHS ---
def get_angle(a, b, c):
    try:
        ba = [a['x'] - b['x'], a['y'] - b['y']]
        bc = [c['x'] - b['x'], c['y'] - b['y']]
        dot = ba[0]*bc[0] + ba[1]*bc[1]
        mag_a = math.hypot(*ba)
        mag_c = math.hypot(*bc)
        return math.degrees(math.acos(max(-1, min(1, dot / (mag_a * mag_c)))))
    except: return 180.0

# --- CONFIGURATION DES ARTICULATIONS MAÎTRESSES ---
MASTER_JOINTS = {
    "squat": ["Hip", "Knee", "Ankle"],
    "shoulder_press": ["Hip", "Shoulder", "Elbow"],
    "bicep_curl": ["Shoulder", "Elbow", "Wrist"],
    "bench_press": ["Shoulder", "Elbow", "Wrist"],
    "lunges": ["Hip", "Knee", "Ankle"]
}

class CalibrationScanner:
    def __init__(self):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.6)
        self.results_log = {}

    def get_view_type(self, lm):
        try:
            s_width = abs(lm['leftShoulder']['x'] - lm['rightShoulder']['x'])
            t_height = abs(lm['leftShoulder']['y'] - lm['leftHip']['y'])
            ratio = s_width / t_height
            return "side" if ratio < 0.45 else "front"
        except: return "unknown"

    def scan_video(self, video_path, exercise_key):
        cap = cv2.VideoCapture(video_path)
        angles_side = []
        angles_front = []
        
        # On essaie de trouver une correspondance dans MASTER_JOINTS
        # Sinon on utilise le genou par défaut
        joints_to_track = MASTER_JOINTS.get(exercise_key, ["Hip", "Knee", "Ankle"])
        
        print(f"  --> Analyse de {os.path.basename(video_path)}...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            
            res = self.pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if res.pose_landmarks:
                lm = {}
                for i, lmk in enumerate(res.pose_landmarks.landmark):
                    name = self.mp_pose.PoseLandmark(i).name.replace('_','').lower()
                    clean = ("left" if "left" in name else "right" if "right" in name else "") + name.replace("left","").replace("right","").capitalize()
                    lm[clean] = {"x": lmk.x, "y": lmk.y, "vis": lmk.visibility}

                view = self.get_view_type(lm)
                side = "left" if lm['leftKnee']['vis'] > lm['rightKnee']['vis'] else "right"
                
                target = [f"{side}{j}" for j in joints_to_track]
                # Sécurité si les landmarks ne sont pas dans le dictionnaire
                if all(t in lm for t in target):
                    current_angle = get_angle(lm[target[0]], lm[target[1]], lm[target[2]])
                    if view == "side": angles_side.append(current_angle)
                    else: angles_front.append(current_angle)

        cap.release()
        
        stats = {}
        if angles_side:
            stats["side"] = {"min": round(min(angles_side),1), "max": round(max(angles_side),1)}
        if angles_front:
            stats["front"] = {"min": round(min(angles_front),1), "max": round(max(angles_front),1)}
        
        return stats

    def run_full_calibration(self, root_folder):
        if not os.path.exists(root_folder):
            print(f"❌ ERREUR : Le dossier {root_folder} est introuvable sur votre Drive.")
            return

        print(f"🚀 DÉMARRAGE DE LA CALIBRATION DRIVE")
        for category in os.listdir(root_folder):
            cat_path = os.path.join(root_folder, category)
            if os.path.isdir(cat_path):
                print(f"\n📂 Catégorie : {category}")
                self.results_log[category] = {}
                
                for exercise in os.listdir(cat_path):
                    exe_path = os.path.join(cat_path, exercise)
                    if os.path.isdir(exe_path):
                        self.results_log[category][exercise] = []
                        for video in os.listdir(exe_path):
                            if video.endswith(('.mp4', '.mov', '.avi', '.MP4')):
                                video_full_path = os.path.join(exe_path, video)
                                result = self.scan_video(video_full_path, exercise.lower())
                                self.results_log[category][exercise].append(result)

        # Sauvegarde du rapport final sur le Drive aussi pour ne pas le perdre
        output_path = os.path.join(root_folder, "calibration_report.json")
        with open(output_path, "w") as f:
            json.dump(self.results_log, f, indent=4)
        print(f"\n✅ CALIBRATION TERMINÉE. Rapport généré dans votre Drive : {output_path}")

# --- EXÉCUTION ---
scanner = CalibrationScanner()
scanner.run_full_calibration(ROOT_DRIVE)

In [ ]:
import cv2
import mediapipe as mp
import math
import json
import os
from google.colab import drive

# --- SETUP ---
drive.mount('/content/drive')
PATH_JSON = '/content/drive/MyDrive/coach_dataset/calibration_report.json'
VIDEO_TEST_INPUT = 'squat_test3.mp4' 
VIDEO_OUTPUT_RESULT = 'results_final_clean.mp4'

def get_angle(a, b, c):
    try:
        ba = [a['x'] - b['x'], a['y'] - b['y']]
        bc = [c['x'] - b['x'], c['y'] - b['y']]
        dot = ba[0]*bc[0] + ba[1]*bc[1]
        mag_a = math.hypot(*ba)
        mag_c = math.hypot(*bc)
        return math.degrees(math.acos(max(-1, min(1, dot / (mag_a * mag_c)))))
    except: return 180.0

class UniversalAnalyzer:
    def __init__(self, exercise_name, calibration_data):
        self.exercise_name = exercise_name.lower()
        self.phase = "up"
        self.reps = 0
        self.was_active = False
        self.status = "READY"
        self.color = (255, 255, 255)
        self.coach_min = 100 
        self.config = self._auto_calibrate(calibration_data)
        
    def _auto_calibrate(self, data):
        found_stats = None
        for cat in data.values():
            if self.exercise_name in cat:
                v_data = cat[self.exercise_name][0]
                found_stats = v_data.get("front") or v_data.get("side")
                break
        if found_stats:
            self.coach_min = found_stats["min"]
            return {
                "down": self.coach_min + 15, 
                "up": found_stats["max"] - 10,
                "joints": ["Hip", "Knee", "Ankle"]
            }
        return {"down": 100, "up": 160, "joints": ["Hip", "Knee", "Ankle"]}

    def analyze(self, lm):
        side = "left" if lm['leftKnee']['vis'] > lm['rightKnee']['vis'] else "right"
        target = [f"{side}{j}" for j in self.config["joints"]]
        angle = get_angle(lm[target[0]], lm[target[1]], lm[target[2]])
        
        # 1. PHASE LOGIC
        if angle < self.config["down"] and self.phase == "up":
            self.phase = "down"
            self.was_active = True
        elif angle > self.config["up"] and self.phase == "down":
            self.phase = "up"
            if self.was_active:
                self.reps += 1
                self.was_active = False

        # 2. STATUS LOGIC: Only "Go Deeper" during DOWN phase
        if self.phase == "down":
            if angle <= self.coach_min + 5:
                self.status = "EXCELLENT"
                self.color = (0, 255, 0)
            elif angle <= self.config["down"]:
                self.status = "GOOD"
                self.color = (0, 255, 255)
            else:
                self.status = "GO DEEPER"
                self.color = (0, 0, 255)
        else:
            # When moving UP or standing still
            self.status = "GREAT REP!" if self.was_active else "READY"
            self.color = (0, 255, 0) if self.was_active else (255, 255, 255)
            
        return self.reps, self.phase

# --- RUNTIME ---
with open(PATH_JSON, 'r') as f:
    calibration_results = json.load(f)

analyzer = UniversalAnalyzer("squat", calibration_results)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5)
cap = cv2.VideoCapture(VIDEO_TEST_INPUT)
w, h, fps = int(cap.get(3)), int(cap.get(4)), cap.get(5)
out = cv2.VideoWriter(VIDEO_OUTPUT_RESULT, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    res = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    if res.pose_landmarks:
        lm = {}
        for i, lmk in enumerate(res.pose_landmarks.landmark):
            name = mp_pose.PoseLandmark(i).name.replace('_','').lower()
            clean = ("left" if "left" in name else "right" if "right" in name else "") + name.replace("left","").replace("right","").capitalize()
            lm[clean] = {"x": lmk.x, "y": lmk.y, "vis": lmk.visibility}

        reps, phase = analyzer.analyze(lm)

        # 3. SEGMENTED UI: NO OVERLAP
        cv2.rectangle(frame, (0,0), (w, 130), (10, 10, 10), -1) 
        
        # ZONE 1: REPS (FAR LEFT)
        cv2.putText(frame, "REPS", (20, 40), 1, 1.2, (150, 150, 150), 2)
        cv2.putText(frame, str(reps), (25, 110), 1, 4, (255, 255, 255), 4)
        
        # ZONE 2: STATUS (CENTERED)
        s_text = analyzer.status
        s_size = cv2.getTextSize(s_text, 1, 3, 4)[0]
        s_x = int((w - s_size[0]) / 2)
        cv2.putText(frame, s_text, (s_x, 80), 1, 3, analyzer.color, 5)
        
        # ZONE 3: PHASE (FAR RIGHT)
        p_text = f"PHASE: {phase.upper()}"
        p_size = cv2.getTextSize(p_text, 1, 1.5, 2)[0]
        cv2.putText(frame, p_text, (w - p_size[0] - 20, 75), 1, 1.5, (180, 180, 180), 2)

        mp.solutions.drawing_utils.draw_landmarks(frame, res.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    out.write(frame)

cap.release()
out.release()
print(f"✅ Clean UI & Logic Fixed: {VIDEO_OUTPUT_RESULT}")